## 📚 Importación de librerías necesarias

Importación de la librería Pandas para el DataWranling y también de la librería MongoClient para la conexión a la BD almacenada en MongoDB

In [ ]:
import pandas as pd
import pymongo as pm
from pymongo import MongoClient as mongo

print(F"✅ ¡Pandas importado correctamente! Versión instalada = {pd.__version__}")
print(F"✅ ¡MongoDB importado correctamente! Versión instalada = {pm.__version__}")

## 🖥️ Conectar a MongoDB y extracción de datos

Utilización del controlador (driver) para conectarme a MongoDB y extraer los datos en un DataFrame.

In [ ]:
# Almacenar la conexión a MongoDB en una variable
cliente = mongo("mongodb://localhost:27017/")

# Seleccionar la base de datos y la colección
db = cliente["calidad_datos"]
coleccion = db["clientes_calidad"]

# Obtener todos los registros dentro de la tabla y guardarlos como una lista de diccionarios
registros = list(coleccion.find())

# Convertir todos los registros a un DataFrame para su tratamiento eliminado el atributo "_id"
df = pd.DataFrame(registros)
df.drop(columns=["_id"], inplace=True)
df.head()

## ✅ 1) DIMENSIÓN: Completitud

Revisar que los datos realmente tengan valores, que estén presentes, ver si hay valores perdidos, nulos, etc.

### 🔸 A. Revisión general de nulos

Esto ayuda a tener una vista rápida de la completitud de los datos.

In [ ]:
# Ver cuántos nulos hay por columna
df.isna().sum()

### 🔸 B. Detectar campos vacíos o en blanco (solo espacios)

Aquí usamos .str.strip() para eliminar espacios en blanco antes de evaluar si el campo está efectivamente vacío.

In [ ]:
# Detectar campos vacíos, o en blanco, que sean solo espacio
df[df["nombre"].isna() | (df["nombre"].str.strip() == "")]

In [ ]:
df[df["email"].isna() | (df["email"].str.strip() == "")]

In [ ]:
df[df["telefono"].isna() | (df["telefono"].astype(str).str.strip() == "")]

### 🔸 C. Identificar registros sin ningún atributo útil

Acá se definen columnas críticas y se buscan registros vacíos en ellas.

In [ ]:
# Identificar registros completos sin ningún atributo útil
# Definir las columnas críticas que voy a evaluar
columnas_criticas = ["nombre","email","telefono","cliente_id"]

# Buscar registros completamente vacíos en estas columnas
df[df[columnas_criticas].isna().all(axis=1)]

## ✅ 2) DIMENSIÓN: Validez

Validación de formatos y tipos.

### 🔸 A. Validación de formato de correo electrónico

Una verificación básica podría ser revisar que el email contenga un @ y un .

In [ ]:
# Validar el formato del correo electrónico
# Filtrar los registros con email NO NULO
emails_no_nulos = df["email"].dropna()

# Aplicar una condición sobre los emails NO NULOS (válidos)
condicion = ~emails_no_nulos.str.contains("@") | ~emails_no_nulos.str.contains(r"\.")

# Mostrar los registros que tengan formato incorrecto
df.loc[emails_no_nulos[condicion].index]

### 🔸 B. Validar formato de teléfono

Esto permite identificar errores como abc123 o 123.

In [ ]:
# Detectar teléfonos que contienen letras o símbolos (deben ser numéricos)
df[df["telefono"].notna() & (~df["telefono"].astype(str).str.isnumeric())]

In [ ]:
# Detectar teléfonos demasiado cortos o largos
df[df["telefono"].notna() & (df["telefono"].astype(str).str.len() < 8)]

### 🔸 C. Validar formato de fechas (fecha_nacimiento y ultima_actualizacion)

Esto detecta fechas con formatos incorrectos como 02/11/1990 si no siguen un patrón ISO.

In [ ]:
# Intentar convertir fecha_nacimiento a datetime
df["fecha_valida"] = pd.to_datetime(df["fecha_nacimiento"], errors="coerce")

# Mostrar registros con fechas inválidas
df[df["fecha_nacimiento"].notna() & df["fecha_valida"].isna()]

In [ ]:
df["ultima_actualizacion_valida"] = pd.to_datetime(df["ultima_actualizacion"], errors="coerce")

df[df["ultima_actualizacion"].notna() & df["ultima_actualizacion_valida"].isna()]

### 🔸 D. Validar valores permitidos en campo estado

Aquí puedes identificar valores como True, 1, "Activo" (con mayúsculas), etc.

In [ ]:
# Mostrar valores únicos para detectar inconsistencias
df["estado"].value_counts(dropna=False)

In [ ]:
# Ver registros con valores inesperados
valores_validos = ["activo", "inactivo"]
df[~df["estado"].astype(str).str.lower().isin(valores_validos)]

## ✅ 3) DIMENSIÓN: Consistencia

Evalúa si los datos tienen sentido lógico o representan correctamente lo que dicen.

### 🔸 A. Validación de consistencia en la región

Verificar valores distintos que significan lo mismo ("RM" vs "Región Metropolitana").

In [ ]:
# Mostrar todos los valores únicos de la columna región
df["region"].value_counts(dropna=False)

In [ ]:
# Identificar registros que podrían tener valores equivalentes en "region"
inconsistencias_region = df[df["region"].isin(["RM", "Región Metropolitana"])]
inconsistencias_region

### 🔸 B. Detectar direcciones escritas de distintas formas

Filtrar registros con direcciones que empiecen con "Av." o "Avenida".

In [ ]:
# Filtrar registros con direcciones que empiecen con "Av." o "Avenida"
inconsistencias_direccion = df[df["direccion"].astype(str).str.contains(r"^(?:Av\.|Avenida)", case=False, na=False)]
inconsistencias_direccion

### 🔸 C. Detectar abreviaturas comunes en direcciones

Buscar patrones de abreviaturas que puedan requerir estandarización futura.

In [ ]:
patrones_direccion = df["direccion"].dropna().str.extract(r"^(?:Av\.|Avenida)", expand=False).value_counts()
patrones_direccion

### 🔸 D. Revisión de valores únicos en estado

Filtra todos los registros únicos del campo estado para detectar potenciales inconsistencias.

In [ ]:
# Ver valores únicos del campo estado
df["estado"].value_counts(dropna=False)

### 🔸 E. Búsqueda de inconsistencias en estado

Detección de valores inconsistentes en el estado de los registros.

In [ ]:
# Queremos detectar casos donde el estado no esté en los valores esperados ("activo" o "inactivo")
# pero que aún así representen el mismo concepto (por ejemplo: True, 1, "Activo")
valores_validos = ["activo", "inactivo"]

# Convertir todo a string en minúsculas para compararlo con los valores esperados
inconsistencias_estado = df[
    df["estado"].notna() &
    ~df["estado"].astype(str).str.lower().isin(valores_validos)
]
inconsistencias_estado

In [ ]:
# Analizar cuántos registros presentan estas inconsistencias
inconsistencias_estado["estado"].value_counts()

## ✅ 4) DIMENSIÓN: Unicidad

Detección de problemas de unicidad en el DataFrame en campos como cliente_id, nombre y email

### 🔸 A. Verificar duplicados en cliente_id

Verificar duplicados en la clave primaria: cliente_id.

In [ ]:
# Verificar duplicados en la clave primaria: cliente_id
duplicados_cliente_id = df[df.duplicated(subset=["cliente_id"], keep=False)]
duplicados_cliente_id

### 🔸 B. Verificar posibles duplicados en nombre

Buscar nombres exactos duplicados.

In [ ]:
# Buscar nombres exactos duplicados
duplicados_nombre_exactos = df[df.duplicated(subset=["nombre"], keep=False)]
duplicados_nombre_exactos

# Opcional: detectar similitudes de nombres (ej. 'Juan Pérez' vs 'Juan Perez')
# Esto no es un duplicado exacto, pero sirve para análisis exploratorio
# (más adelante en Wrangling se podrían usar librerías como fuzzywuzzy)
nombres_similares = df["nombre"].str.lower().value_counts()
nombres_similares[nombres_similares > 1]

### 🔸 C. Verificar posibles duplicados en email

Buscar email exactos duplicados.

In [ ]:
# Buscar duplicados exactos de email
duplicados_email = df[df.duplicated(subset=["email"], keep=False) & df["email"].notna()]
duplicados_email

## ✅ 5) DIMENSIÓN: Actualidad

Revisión de los datos con fechas para revisar la actualidad de los registros.

### 🔸 A. Tratamiento del campo 'ultima_actualizacion' previo al análisis

Convertir la columna 'ultima_actualizacion' a datetime.

In [ ]:
from datetime import datetime

# Convertir la columna 'ultima_actualizacion' a datetime
df["ultima_actualizacion_valida"] = pd.to_datetime(df["ultima_actualizacion"], errors="coerce")

# Definir umbral de "desactualización" (por ejemplo: antes de 2018)
fecha_umbral = pd.Timestamp("2018-01-01")

# Filtrar registros desactualizados
registros_desactualizados = df[df["ultima_actualizacion_valida"].notna() & (df["ultima_actualizacion_valida"] < fecha_umbral)]
registros_desactualizados

### 🔸 B. Calcular la antiguedad de los registros

Determinar en años la antiguedad de aquellos registros con datos.

In [ ]:
# Calcular la antigüedad en años de cada registro
df["antiguedad_ultima_actualizacion"] = (datetime.now() - df["ultima_actualizacion_valida"]).dt.days / 365
df[["cliente_id", "nombre", "ultima_actualizacion", "antiguedad_ultima_actualizacion"]]

In [ ]:
# Identificar registros sin fecha de actualización (posiblemente nunca actualizados)
registros_sin_actualizacion = df[df["ultima_actualizacion_valida"].isna()]
registros_sin_actualizacion

## 🛠️ Plan de Data Wrangling

### 1️⃣ Completitud
Problema: Campos vacíos o nulos en nombre, email, telefono y registros con información incompleta.

### 2️⃣ Validez
Problema: Formatos incorrectos en email, telefono y fechas.

### 3️⃣ Consistencia
Problema: Valores que significan lo mismo pero con formatos distintos ("RM" vs "Región Metropolitana", "Activo" vs True).

### 4️⃣ Unicidad
Problema: Duplicados en cliente_id, nombre y email.

### 5️⃣ Actualidad
Problema: Registros sin fecha de actualización o con datos antiguos.

## 🛠 Data Wrangling

### 1️⃣ Completitud

In [ ]:
# Reemplazar valores nulos o vacíos con texto estándar
df["nombre"] = df["nombre"].fillna("desconocido").str.strip()
df["email"] = df["email"].fillna("no_disponible").str.strip()
df["telefono"] = df["telefono"].fillna("no_disponible").astype(str).str.strip()

# Reemplazar valores vacíos (solo espacios) por "desconocido" o "no_disponible"
df.loc[df["nombre"] == "", "nombre"] = "desconocido"
df.loc[df["email"] == "", "email"] = "no_disponible"
df.loc[df["telefono"] == "", "telefono"] = "no_disponible"

# Identificar registros sin datos críticos (cliente_id vacío)
registros_sin_cliente_id = df[df["cliente_id"].isna()]
if not registros_sin_cliente_id.empty:
    print("Registros sin cliente_id detectados:", len(registros_sin_cliente_id))

## 🛠 Data Wrangling

### 2️⃣ Validez

In [ ]:
import re

# Normalizar emails inválidos
patron_email = r"^[\w\.-]+@[\w\.-]+\.\w+$"
df.loc[~df["email"].str.match(patron_email, na=False), "email"] = "email_invalido"

# Limpiar teléfonos no numéricos o absurdos
df.loc[~df["telefono"].str.isnumeric(), "telefono"] = "telefono_invalido"
df.loc[df["telefono"].str.len() < 8, "telefono"] = "telefono_invalido"
df.loc[df["telefono"].str.len() > 12, "telefono"] = "telefono_invalido"

# Normalizar fechas de nacimiento y última actualización
df["fecha_nacimiento"] = pd.to_datetime(df["fecha_nacimiento"], errors="coerce")
df["ultima_actualizacion"] = pd.to_datetime(df["ultima_actualizacion"], errors="coerce")

# Validar rut simple (si existe columna rut)
if "rut" in df.columns:
    df.loc[~df["rut"].astype(str).str.match(r"^\d{1,2}\.\d{3}\.\d{3}-[0-9KkXx]$", na=False), "rut"] = "rut_invalido"

## 🛠 Data Wrangling

### 3️⃣ Consistencia

In [ ]:
# Normalizar región
region_map = {
    "RM": "Región Metropolitana",
    "rm": "Región Metropolitana"
}
df["region"] = df["region"].replace(region_map)

# Normalizar estado
estado_map = {
    "activo": "activo",
    "inactivo": "inactivo",
    "true": "activo",
    "1": "activo",
    "false": "inactivo",
    "0": "inactivo"
}
df["estado"] = df["estado"].astype(str).str.lower().replace(estado_map)

# Estandarizar direcciones ("Av." -> "Avenida")
df["direccion"] = df["direccion"].astype(str).str.replace(r"^Av\.", "Avenida", case=False, regex=True)

## 🛠 Data Wrangling

### 4️⃣ Unicidad

In [ ]:
# Eliminar duplicados en cliente_id conservando el primero
df = df.drop_duplicates(subset=["cliente_id"], keep="first")

# Detectar duplicados exactos de email y marcarlos
duplicados_email = df[df.duplicated(subset=["email"], keep=False) & df["email"].notna()]
if not duplicados_email.empty:
    print("⚠️ Emails duplicados detectados:", duplicados_email["email"].nunique())

# Detectar posibles duplicados de nombre
duplicados_nombre = df[df.duplicated(subset=["nombre"], keep=False)]
if not duplicados_nombre.empty:
    print("⚠️ Nombres duplicados detectados:", duplicados_nombre["nombre"].nunique())

## 🛠 Data Wrangling

### 5️⃣ Actualidad

In [ ]:
# Crear columna para marcar registros desactualizados
fecha_umbral = pd.Timestamp("2018-01-01")
df["desactualizado"] = df["ultima_actualizacion"].apply(lambda x: "sí" if pd.notna(x) and x < fecha_umbral else "no")

# Marcar registros sin fecha
df["desactualizado"] = df["desactualizado"].mask(df["ultima_actualizacion"].isna(), "sin_actualizacion")

# Recalcular antigüedad en años
df["antiguedad_ultima_actualizacion"] = (pd.Timestamp.now() - df["ultima_actualizacion"]).dt.days / 365